# Прогнозирование с помощью регрессии

In [1]:
%pylab inline
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error
import numpy as np
from datetime import timedelta

Populating the interactive namespace from numpy and matplotlib


Объявляем список регионов для которых будем строить прогноз и загружаем аггрегированные по часам данные для этих регионов



In [2]:
region_list = [1075, 1076, 1077, 1125, 1126, 1127, 1128, 1129, 1130, 1131, 1132, 1172, 1173, 1174, 1175, 1176, 1177,
               1178, 1179, 1180, 1181, 1182, 1183, 1184, 1221, 1222, 1223, 1224, 1225, 1227, 1228, 1229, 1230, 1231,
               1232, 1233, 1234, 1235, 1272, 1273, 1274, 1278, 1279, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287,
               1326, 1327, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1376, 1377, 1378, 1380, 1382, 1383,
               1384, 1385, 1386, 1387, 1388, 1389, 1390, 1426, 1431, 1434, 1435, 1436, 1437, 1438, 1439, 1441, 1442,
               1480, 1482, 1483, 1530, 1532, 1533, 1580, 1630, 1684, 1733, 1734, 1783, 2068, 2069, 2118, 2119, 2168]
data = pd.read_csv('regions_counts_by_hours.csv', index_col=0)
data.datetime = data.datetime.map(lambda x: pd.to_datetime(x))

train_from = pd.to_datetime('2016-02-01 00:00:00')
train_to = pd.to_datetime('2016-04-30 17:00:00')

test_from = pd.to_datetime('2016-04-30 23:00:00')
test_to = pd.to_datetime('2016-05-31 17:00:00')

Создаём набор признаков

In [3]:
X_fields = {'count', 'region', 'day', 'month', 'weekday', 'hour', 'sum_p_12_h', 'sum_p_24_h', 'sum_p_w'}
y_fields = set()
values = pd.DataFrame()
for region in region_list:
    value = pd.DataFrame()
    reg = str(region)
    value['count'] = data[reg]
    value['region'] = [region] * data.shape[0]
    value['datetime'] = data['datetime']

    # Добавляем признаки дня, месяца, дня недели и часа. Поскольку у меня данные за 1 год, то признак года добавлять смысла нет.
    value['day'] = data.datetime.map(lambda x: x.day)
    value['month'] = data.datetime.map(lambda x: x.month)
    value['weekday'] = data.datetime.map(lambda x: x.weekday())
    value['hour'] = data.datetime.map(lambda x: x.hour)

    # Добавляем признаки числа поездок из этого региона за прошлые часы и дни
    for i in xrange(1, 24):
        value['ph_' + str(i)] = data[reg] - data[reg].shift(i)
        X_fields.add('ph_' + str(i))
      
    for i in xrange(1, 3):
        value['pd_' + str(i)] = data[reg] - data[reg].shift(24*i)
        X_fields.add('pd_' + str(i))

    value['sum_p_12_h'] = [value['count'][i-12:i].sum() for i in xrange(data.shape[0])]
    value['sum_p_24_h'] = [value['count'][i-24:i].sum() for i in xrange(data.shape[0])]
    value['sum_p_w'] = [value['count'][i-24*7:i].sum() for i in xrange(data.shape[0])]
    
    # Добавляем целевые значения для каждой из 6 можделей
    for i in xrange(1, 7):
        value['target_' + str(i)] = data[reg].shift(-i)
        y_fields.add('target_' + str(i))
        
    values = values.append(value)

In [4]:
values.head()

,count,region,datetime,day,month,weekday,hour,ph_1,ph_2,ph_3,...,pd_2,sum_p_12_h,sum_p_24_h,sum_p_w,target_1,target_2,target_3,target_4,target_5,target_6
0,80,1075,2016-01-01 00:00:00,1,1,4,0,NaN,NaN,NaN,...,NaN,0,0,0,91.0,90.0,32.0,24.0,11.0,7.0
1,91,1075,2016-01-01 01:00:00,1,1,4,1,11.0,NaN,NaN,...,NaN,0,0,0,90.0,32.0,24.0,11.0,7.0,9.0
2,90,1075,2016-01-01 02:00:00,1,1,4,2,-1.0,10.0,NaN,...,NaN,0,0,0,32.0,24.0,11.0,7.0,9.0,18.0
3,32,1075,2016-01-01 03:00:00,1,1,4,3,-58.0,-59.0,-48.0,...,NaN,0,0,0,24.0,11.0,7.0,9.0,18.0,22.0
4,24,1075,2016-01-01 04:00:00,1,1,4,4,-8.0,-66.0,-67.0,...,NaN,0,0,0,11.0,7.0,9.0,18.0,22.0,27.0


Подбираем значения параметра alpha для каждой из моделей

In [5]:
alphas = np.arange(5, 100, 5)
topModels = {}
topAlphas = {}

X_fields = list(X_fields)
y_fields = list(y_fields)

models = dict()

# Для каждой модели на данных до апреля включительно подберём параметр альфа
for i in xrange(1, 7):
    best_model, best_alpha, best_mae = None, None, float('inf')
    
    for alpha in alphas:
        model = Lasso(alpha=alpha)
        
        train = values[(values.datetime >= train_from) & (values.datetime <= train_to)]
        test = values[(values.datetime >= test_from) & (values.datetime <= test_to)]
        

        model_fitted = model.fit(train[X_fields].values, train['target_' + str(i)].values)
        predict = model_fitted.predict(test[X_fields].values)
        mae = mean_absolute_error(test['target_' + str(i)], predict)
        
        if mae < best_mae:
            best_mae = mae
            best_model = model_fitted
            best_alpha = alpha
            
    models[i] = (best_model, best_alpha)

/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [6]:
models

{1: (Lasso(alpha=5, copy_X=True, fit_intercept=True, max_iter=1000,
     normalize=False, positive=False, precompute=False, random_state=None,
     selection='cyclic', tol=0.0001, warm_start=False), 5),
 2: (Lasso(alpha=5, copy_X=True, fit_intercept=True, max_iter=1000,
     normalize=False, positive=False, precompute=False, random_state=None,
     selection='cyclic', tol=0.0001, warm_start=False), 5),
 3: (Lasso(alpha=10, copy_X=True, fit_intercept=True, max_iter=1000,
     normalize=False, positive=False, precompute=False, random_state=None,
     selection='cyclic', tol=0.0001, warm_start=False), 10),
 4: (Lasso(alpha=15, copy_X=True, fit_intercept=True, max_iter=1000,
     normalize=False, positive=False, precompute=False, random_state=None,
     selection='cyclic', tol=0.0001, warm_start=False), 15),
 5: (Lasso(alpha=15, copy_X=True, fit_intercept=True, max_iter=1000,
     normalize=False, positive=False, precompute=False, random_state=None,
     selection='cyclic', tol=0.0001, war

Уточняем значение alpha

In [7]:
for i in xrange(1, 7):
    best_model, best_alpha, best_mae = None, None, float('inf')
    alpha = models[i][1]
    
    for a in xrange(alpha if alpha == 5 else alpha - 4, alpha + 5):
        model = Lasso(alpha=a)
        
        train = values[(values.datetime >= train_from) & (values.datetime <= train_to)]
        test = values[(values.datetime >= test_from) & (values.datetime <= test_to)]
        

        model_fitted = model.fit(train[X_fields].values, train['target_' + str(i)].values)
        predict = model_fitted.predict(test[X_fields].values)
        mae = mean_absolute_error(test['target_' + str(i)], predict)
        
        if mae < best_mae:
            best_mae = mae
            best_model = model_fitted
            best_alpha = alpha
            
    models[i] = (best_model, best_alpha)

In [8]:
models

{1: (Lasso(alpha=5, copy_X=True, fit_intercept=True, max_iter=1000,
     normalize=False, positive=False, precompute=False, random_state=None,
     selection='cyclic', tol=0.0001, warm_start=False), 5),
 2: (Lasso(alpha=5, copy_X=True, fit_intercept=True, max_iter=1000,
     normalize=False, positive=False, precompute=False, random_state=None,
     selection='cyclic', tol=0.0001, warm_start=False), 5),
 3: (Lasso(alpha=9, copy_X=True, fit_intercept=True, max_iter=1000,
     normalize=False, positive=False, precompute=False, random_state=None,
     selection='cyclic', tol=0.0001, warm_start=False), 10),
 4: (Lasso(alpha=13, copy_X=True, fit_intercept=True, max_iter=1000,
     normalize=False, positive=False, precompute=False, random_state=None,
     selection='cyclic', tol=0.0001, warm_start=False), 15),
 5: (Lasso(alpha=15, copy_X=True, fit_intercept=True, max_iter=1000,
     normalize=False, positive=False, precompute=False, random_state=None,
     selection='cyclic', tol=0.0001, warm

Выбранными моделями построим для каждой географической зоны и каждого конца истории от 2016.04.30 23:00 до 2016.05.31 17:00 прогнозы на 6 часов вперёд; посчитайте в ноутбуке ошибку прогноза

In [37]:
_from = pd.to_datetime('2016-04-30 23:00:00')

total_mae = 0
for region in region_list:
    mae = 0  
    items = values[values['region'] == region]
    
    for j in range(0, 739):
        pred_datetime = _from + timedelta(hours=j)
        current_data = items[items['datetime'] == pred_datetime]
        
        mae += np.sum(
            mean_absolute_error(current_data['target_' + str(i)], models[i][0].predict(current_data[X_fields]))
            for i in range(1, 7)
        )
    total_mae += mae

In [38]:
print total_mae / (102 * 739 * 6)

31.8420381038


Ошибка с прошлой недели сильно уменьшилась. На прошлой неделе получалась в районе 37.5.

Итоговыми моделями постройте прогнозы для каждого конца истории от 2016.05.31 23:00 до 2016.06.30 17:00 и запишите все результаты в один файл в формате geoID, histEndDay, histEndHour, step, y. Здесь geoID — идентификатор зоны, histEndDay — день конца истории в формате id,y, где столбец id состоит из склеенных через подчёркивание идентификатора географической зоны, даты конца истории, часа конца истории и номера отсчёта, на который делается предсказание (1-6); столбец y — ваш прогноз.



In [13]:
_from = pd.to_datetime('2016-02-01 00:00:00')

results = open('results.csv', 'w')
results.write('id,y\n')
for region in region_list:
    mae = 0  
    items = values[values['region'] == region]
    
    for j in range(0, 3623):
        pred_datetime = _from + timedelta(hours=j)
        current_data = items[items['datetime'] == pred_datetime]
        
        for i in range(1, 7):
            predict = models[i][0].predict(current_data[X_fields])
            results.write('{}_{}_{}_{},{}\n'.format(region, pred_datetime.date(), pred_datetime.hour, i, predict[0]))
    
results.close()

https://www.kaggle.com/submissions/7593650/7593650.zip

Мой результат: https://yadi.sk/i/AP5PfiVd3WpBcA (видно что он сильно лучше предыдущего результата).